In [1]:
from utils import plot_ship_trajectory
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Cell 1: Imports & βασικές ρυθμίσεις

# Path για το CSV σου
CSV_PATH = "data/ais_data_5min_clean.csv"  # άλλαξέ το αν χρειάζεται

# Hyperparameters για τα παράθυρα
INPUT_LEN = 20     # πόσα ιστορικά σημεία βάζουμε στο input
OUTPUT_LEN = 6     # πόσα μελλοντικά σημεία προβλέπουμε

# Split by MMSI
TRAIN_FRAC = 0.7
VAL_FRAC   = 0.15
TEST_FRAC  = 0.15

RANDOM_STATE = 42

# Model hyperparameters
INPUT_DIM  = 6   # [Latitude, Longtitude, SOG, COG, speed_mps_track]
OUTPUT_DIM = 5   # [Latitude, Longtitude]
HIDDEN_DIM = 128
NUM_LAYERS = 2
DROPOUT    = 0.1
BATCH_SIZE = 128
LR         = 1e-3
NUM_EPOCHS = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [3]:
# Cell 2A: Split the original CSV into train/val/test CSVs by MMSI and save them

import os
import numpy as np
import pandas as pd

def split_and_save_csv_by_mmsi(
    full_csv_path: str,
    train_csv_path: str,
    val_csv_path: str,
    test_csv_path: str,
    train_frac: float = 0.7,
    val_frac: float = 0.15,
    test_frac: float = 0.15,
    random_state: int = 42,
):
    df = pd.read_csv(full_csv_path)

    # Example function to apply to your raw DataFrame before splitting/scaling
    def apply_cog_encoding(df: pd.DataFrame):
        # 1. Convert COG from degrees to radians
        df['COG_rad'] = np.deg2rad(df['COG'])

        # 2. Create the two new cyclical features
        df['COG_sin'] = np.sin(df['COG_rad'])
        df['COG_cos'] = np.cos(df['COG_rad'])

        # 3. Remove the original COG and temporary radian columns
        # Use errors='ignore' in case COG was already dropped in a previous run
        df = df.drop(columns=['COG', 'COG_rad'], errors='ignore')
        return df

    # Ensure Timestamp is datetime
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    
    # <<<--- 1. INSERT THE FUNCTION CALL HERE --->>>
    df = apply_cog_encoding(df)

    # 2. UPDATE THE REQUIRED COLUMNS LIST (Remove 'COG', Add 'COG_sin', 'COG_cos')
    required_cols = [
        "Timestamp", "MMSI", "SOG", 
        "Longtitude", "Latitude",
        "COG_sin", "COG_cos" # <-- New features for checking
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print("WARNING: Missing columns:", missing)

    df = df.dropna(subset=["MMSI", "Timestamp", "Latitude", "Longtitude"])

    # Shuffle MMSIs and split
    rng = np.random.RandomState(random_state)
    unique_mmsi = df["MMSI"].unique()
    rng.shuffle(unique_mmsi)

    n = len(unique_mmsi)
    n_train = int(n * train_frac)
    n_val   = int(n * val_frac)

    mmsi_train = unique_mmsi[:n_train]
    mmsi_val   = unique_mmsi[n_train:n_train + n_val]
    mmsi_test  = unique_mmsi[n_train + n_val:]

    df_train = df[df["MMSI"].isin(mmsi_train)].copy()
    df_val   = df[df["MMSI"].isin(mmsi_val)].copy()
    df_test  = df[df["MMSI"].isin(mmsi_test)].copy()

    # Sort each split nicely
    df_train = df_train.sort_values(["MMSI", "Timestamp"])
    df_val   = df_val.sort_values(["MMSI", "Timestamp"])
    df_test  = df_test.sort_values(["MMSI", "Timestamp"])

    # Create folder if needed
    os.makedirs(os.path.dirname(train_csv_path) or ".", exist_ok=True)

    # These lines save the files with the new COG columns
    df_train.to_csv(train_csv_path, index=False)
    df_val.to_csv(val_csv_path,   index=False)
    df_test.to_csv(test_csv_path, index=False)

    print(f"Saved train CSV to: {train_csv_path} (rows: {len(df_train)})")
    print(f"Saved val   CSV to: {val_csv_path} (rows: {len(df_val)})")
    print(f"Saved test  CSV to: {test_csv_path} (rows: {len(df_test)})")
    print(f"# MMSI -> train: {len(mmsi_train)}, val: {len(mmsi_val)}, test: {len(mmsi_test)}")

    # Optionally, also save MMSI lists separately if you want
    np.save("mmsi_train.npy", mmsi_train)
    np.save("mmsi_val.npy",   mmsi_val)
    np.save("mmsi_test.npy",  mmsi_test)
    print("Saved mmsi_train.npy, mmsi_val.npy, mmsi_test.npy")

    return df_train, df_val, df_test, mmsi_train, mmsi_val, mmsi_test

In [4]:
# Choose output file names (you can adjust paths if you want)
TRAIN_CSV = "ais_train.csv"
VAL_CSV   = "ais_val.csv"
TEST_CSV  = "ais_test.csv"

df_train_raw, df_val_raw, df_test_raw, mmsi_train, mmsi_val, mmsi_test = split_and_save_csv_by_mmsi(
    full_csv_path=CSV_PATH,      # your original full AIS CSV
    train_csv_path=TRAIN_CSV,
    val_csv_path=VAL_CSV,
    test_csv_path=TEST_CSV,
    train_frac=TRAIN_FRAC,
    val_frac=VAL_FRAC,
    test_frac=TEST_FRAC,
    random_state=RANDOM_STATE,
)


Saved train CSV to: ais_train.csv (rows: 50033)
Saved val   CSV to: ais_val.csv (rows: 9760)
Saved test  CSV to: ais_test.csv (rows: 11528)
# MMSI -> train: 198, val: 42, test: 43
Saved mmsi_train.npy, mmsi_val.npy, mmsi_test.npy


In [5]:
# Cell 3: Prepare datasets from already-split train/val/test CSV files

def prepare_datasets_from_split_csvs(
    train_csv: str,
    val_csv: str,
    test_csv: str,
    input_len: int = 20,
    output_len: int = 5,
):
    # Load split CSVs
    df_train = pd.read_csv(train_csv)
    df_val   = pd.read_csv(val_csv)
    df_test  = pd.read_csv(test_csv)

    # Make sure Timestamp is datetime type
    for df in [df_train, df_val, df_test]:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"])

    # Sort them
    df_train = df_train.sort_values(["MMSI", "Segment", "Timestamp"])
    df_val   = df_val.sort_values(["MMSI", "Segment", "Timestamp"])
    df_test  = df_test.sort_values(["MMSI", "Segment", "Timestamp"])

# --- WITH THIS NEW LINE ---
    feature_cols_in  = [
        "Latitude", 
        "Longtitude", 
        "SOG", 
        "speed_mps_track", 
        "COG_sin",   # <--- New
        "COG_cos"    # <--- New
    ]
    feature_cols_out = ["Latitude", 
    "Longtitude",
    "SOG",          # <--- New output feature
    "COG_sin",      # <--- New output feature
    "COG_cos"]
    cols_to_scale = list(set(feature_cols_in + feature_cols_out))

    # ---- Fit scaler on TRAIN ONLY ----
    means = {}
    stds  = {}
    for col in cols_to_scale:
        m = df_train[col].mean()
        s = df_train[col].std()
        if s == 0 or np.isnan(s):
            s = 1.0
        means[col] = float(m)
        stds[col]  = float(s)

    scaler = {"mean": means, "std": stds}

    def normalize_df(df_subset):
        df_norm = df_subset.copy()
        for col in cols_to_scale:
            df_norm[col] = (df_norm[col] - means[col]) / stds[col]
        return df_norm

    df_train_n = normalize_df(df_train)
    df_val_n   = normalize_df(df_val)
    df_test_n  = normalize_df(df_test)

    # ---- Build sliding-window sequences per (MMSI, Segment) ----
    def build_sequences(df_subset_norm):
        X_list, Y_list = [], []
        total_len = input_len + output_len

        for (mmsi, seg_id), seg_df in df_subset_norm.groupby(["MMSI", "Segment"]):
            seg_values_in  = seg_df[feature_cols_in].to_numpy(dtype=np.float32)
            seg_values_out = seg_df[feature_cols_out].to_numpy(dtype=np.float32)

            n_pts = len(seg_df)
            if n_pts < total_len:
                continue

            for start in range(0, n_pts - total_len + 1):
                in_slice  = seg_values_in[start : start + input_len]              # (L, 5)
                out_slice = seg_values_out[start + input_len : start + total_len] # (H, 2)
                X_list.append(in_slice)
                Y_list.append(out_slice)

        if not X_list:
            X_arr = np.empty((0, input_len, len(feature_cols_in)), dtype=np.float32)
            Y_arr = np.empty((0, output_len, len(feature_cols_out)), dtype=np.float32)
        else:
            X_arr = np.stack(X_list, axis=0)
            Y_arr = np.stack(Y_list, axis=0)

        return X_arr, Y_arr

    X_train, Y_train = build_sequences(df_train_n)
    X_val,   Y_val   = build_sequences(df_val_n)
    X_test,  Y_test  = build_sequences(df_test_n)

    print(f"Train samples: X{X_train.shape}, Y{Y_train.shape}")
    print(f"Val   samples: X{X_val.shape},   Y{Y_val.shape}")
    print(f"Test  samples: X{X_test.shape},  Y{Y_test.shape}")

    return (X_train, Y_train), (X_val, Y_val), (X_test, Y_test), scaler


In [6]:
# Use the three CSVs instead of the big original file

(train_X, train_Y), (val_X, val_Y), (test_X, test_Y), scaler = prepare_datasets_from_split_csvs(
    train_csv=TRAIN_CSV,
    val_csv=VAL_CSV,
    test_csv=TEST_CSV,
    input_len=INPUT_LEN,
    output_len=OUTPUT_LEN,
)


Train samples: X(43883, 20, 6), Y(43883, 6, 5)
Val   samples: X(8610, 20, 6),   Y(8610, 6, 5)
Test  samples: X(10303, 20, 6),  Y(10303, 6, 5)


In [7]:
# Cell 4: Dataset & DataLoaders

class TrajectoryDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = TrajectoryDataset(train_X, train_Y)
val_dataset   = TrajectoryDataset(val_X,   val_Y)
test_dataset  = TrajectoryDataset(test_X,  test_Y)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False)

len(train_loader), len(val_loader), len(test_loader)


(343, 68, 81)

In [8]:
# Cell 5: Ορισμός LSTM Encoder–Decoder (Updated for INPUT_DIM=6, OUTPUT_DIM=5)
import torch.nn as nn

class LSTMEncoderDecoder(nn.Module):
    # input_dim=6 (Lat, Lon, SOG, speed, COG_sin, COG_cos)
    # output_dim=5 (Lat, Lon, SOG, COG_sin, COG_cos)
    def __init__(self, input_dim=6, output_dim=5,
                 hidden_dim=128, num_layers=2, dropout=0.1):
        super().__init__()

        # Encoder must accept input_dim=6
        self.encoder = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        # Decoder must accept input_dim=6 (The full input vector for the next step)
        self.decoder = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        # Output FC layer produces output_dim=5 features (Lat, Lon, SOG, COG_sin, COG_cos)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        """
        src: (B, L, 6)   -- Input sequence (History)
        tgt: (B, H, 5)   -- Target sequence (True Future: Lat/Lon/SOG/COG_sin/COG_cos)
        returns: (B, H, 5) -- Predicted sequence
        """
        batch_size, H, out_dim = tgt.shape # out_dim = 5

        # 1. Encode history
        _, (h, c) = self.encoder(src)

        # 2. First decoder input = full LAST historical vector
        # dec_input must start as (B, 1, 6)
        dec_input = src[:, -1, :].unsqueeze(1) 

        outputs = []
        for t in range(H):
            # Decoder step
            dec_out, (h, c) = self.decoder(dec_input, (h, c))
            step_output = self.fc_out(dec_out) # (B, 1, 5) -> Lat/Lon/SOG/sin/cos
            outputs.append(step_output)

            # Teacher forcing logic
            if self.training and torch.rand(1).item() < teacher_forcing_ratio:
                # Use ALL 5 ground truth features for the next step
                next_input_features = tgt[:, t:t+1, :]   # (B, 1, 5) ground truth
            else:
                # Use ALL 5 predicted features for the next step
                next_input_features = step_output        # (B, 1, 5) prediction

            # 3. Update next input for decoder (Autoregressive Step)
            # dec_input must maintain 6 features: Lat/Lon/SOG/speed_track/COG_sin/COG_cos
            # The next input uses the PREDICTED/TRUE values for Lat/Lon/SOG/COG_sin/COG_cos (5 features)
            # and keeps the speed_mps_track feature from the previous time step.
            
            dec_input = dec_input.clone()        # (B, 1, 6)
            
            # Update the first 3 elements (Lat, Lon, SOG)
            dec_input[:, :, :3] = next_input_features[:, :, :3] # Update Lat, Lon, SOG
            
            # Update the last 2 elements (COG_sin, COG_cos)
            dec_input[:, :, 4:6] = next_input_features[:, :, 3:5] # Update COG_sin/cos (skipping index 3, which is speed_mps_track)


        outputs = torch.cat(outputs, dim=1)  # (B, H, 5)
        return outputs

In [9]:
# Cell 6: Δημιουργία μοντέλου, loss & optimizer

# ----------------------------------------------------
# 1. DEFINE THE HAVERSINE LOSS CLASS
# ----------------------------------------------------
class HaversineLoss(torch.nn.Module):
    def __init__(self, lat_mean, lat_std, lon_mean, lon_std, R=6371000.0):
        super().__init__()
        # Store denormalization constants and Earth radius (R)
        # Use torch.tensor for constants that might be used in calculations
        self.lat_mean = torch.tensor(lat_mean, dtype=torch.float32)
        self.lat_std  = torch.tensor(lat_std, dtype=torch.float32)
        self.lon_mean = torch.tensor(lon_mean, dtype=torch.float32)
        self.lon_std  = torch.tensor(lon_std, dtype=torch.float32)
        self.R = R
        # Use nn.L1Loss as a base to aggregate the distances (Mean Absolute Error)
        self.agg_loss = torch.nn.L1Loss(reduction='mean')

    def denormalize(self, norm_lat, norm_lon):
        # Denormalize back to degrees and move to the device of the input tensor
        device = norm_lat.device
        lat = norm_lat * self.lat_std.to(device) + self.lat_mean.to(device)
        lon = norm_lon * self.lon_std.to(device) + self.lon_mean.to(device)
        # Convert to radians for Haversine formula
        return torch.deg2rad(lat), torch.deg2rad(lon)

    def forward(self, pred, target):
        """
        Calculates the Haversine distance between prediction and target.
        pred, target shape: (B, H, 2), where 2 is (normalized_lat, normalized_lon)
        """
        # --- 1. Denormalize and convert to radians ---
        pred_lat_rad, pred_lon_rad = self.denormalize(pred[..., 0], pred[..., 1])
        true_lat_rad, true_lon_rad = self.denormalize(target[..., 0], target[..., 1])

        # --- 2. Haversine Formula ---
        dlat = true_lat_rad - pred_lat_rad
        dlon = true_lon_rad - pred_lon_rad

        a = torch.sin(dlat / 2.0)**2 + \
            torch.cos(pred_lat_rad) * torch.cos(true_lat_rad) * torch.sin(dlon / 2.0)**2

        # Clamp 'a' to [0, 1] to prevent issues with arcsin(sqrt(a))
        a = torch.clamp(a, 0.0, 1.0)
        
        c = 2.0 * torch.asin(torch.sqrt(a))
        
        # Distance in meters
        distance_m = self.R * c

        # --- 3. Aggregate Loss (Mean Absolute Error of Distance) ---
        return self.agg_loss(distance_m, torch.zeros_like(distance_m))


# ----------------------------------------------------
# 2. MODEL AND LOSS INSTANTIATION
# ----------------------------------------------------

# (Model definition using the class defined in Cell 5)
model = LSTMEncoderDecoder(
    input_dim=INPUT_DIM,
    output_dim=OUTPUT_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
).to(device)


# Extract denormalization constants from your already calculated scaler (from Cell 6/7)
lat_mean = scaler["mean"]["Latitude"]
lat_std  = scaler["std"]["Latitude"]
lon_mean = scaler["mean"]["Longtitude"]
lon_std  = scaler["std"]["Longtitude"]

# Instantiate the Haversine Loss, passing the necessary scaling constants
criterion = HaversineLoss(
    lat_mean=lat_mean,
    lat_std=lat_std,
    lon_mean=lon_mean,
    lon_std=lon_std,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print(model)

LSTMEncoderDecoder(
  (encoder): LSTM(6, 128, num_layers=2, batch_first=True, dropout=0.1)
  (decoder): LSTM(6, 128, num_layers=2, batch_first=True, dropout=0.1)
  (fc_out): Linear(in_features=128, out_features=5, bias=True)
)


In [10]:
import os
import torch

# --------------------------
# CONFIG
# --------------------------
NUM_EPOCHS = 10 # run for 10 epochs

# 👉 Change this to whatever path/filename you want:
# e.g. "/content/drive/MyDrive/deep learning/models/lstm_best.pth"
MODEL_PATH = "data/lstm_best_with_5min.pth"

# Create directory if it does not exist
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

# --------------------------
# TRAINING LOOP WITH BEST-MODEL SAVING
# --------------------------
best_val_loss = float("inf")

for epoch in range(NUM_EPOCHS):
  # ----- TRAIN -----
  model.train()
  total_train_loss = 0.0

  for X_batch, Y_batch in train_loader:
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)

    optimizer.zero_grad()
    preds = model(X_batch, Y_batch, teacher_forcing_ratio=0.5)
    loss = criterion(preds, Y_batch)
    loss.backward()
    optimizer.step()

    total_train_loss += loss.item() * X_batch.size(0)

  avg_train_loss = total_train_loss / len(train_loader.dataset)

  # ----- VALIDATION -----
  model.eval()
  total_val_loss = 0.0

  with torch.no_grad():
    for X_batch, Y_batch in val_loader:
      X_batch = X_batch.to(device)
      Y_batch = Y_batch.to(device)

      preds = model(X_batch, Y_batch, teacher_forcing_ratio=0.0)
      loss = criterion(preds, Y_batch)
      total_val_loss += loss.item() * X_batch.size(0)

  avg_val_loss = total_val_loss / len(val_loader.dataset)

  # ----- SAVE BEST MODEL -----
  if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss

    torch.save({
      "model_state_dict": model.state_dict(),
      "scaler": scaler,
      "input_len": INPUT_LEN,
      "output_len": OUTPUT_LEN,
    }, MODEL_PATH)

    # CHANGE 1: Update print statement to Haversine Loss (m)
    print(f"🔥 New best model saved to {MODEL_PATH}! Val Haversine Loss (m) = {best_val_loss:.4f}")

  # ----- PRINT EPOCH RESULTS -----
  # CHANGE 2: Update print statement labels and formatting
  print(
    f"Epoch {epoch+1}/{NUM_EPOCHS} - "
    f"Train Haversine Loss (m): {avg_train_loss:.4f} - Val Haversine Loss (m): {avg_val_loss:.4f} "
    f"(Best Val: {best_val_loss:.4f})"
  )

🔥 New best model saved to data/lstm_best_with_5min.pth! Val Haversine Loss (m) = 3393.0751
Epoch 1/10 - Train Haversine Loss (m): 14837.7910 - Val Haversine Loss (m): 3393.0751 (Best Val: 3393.0751)
Epoch 2/10 - Train Haversine Loss (m): 4253.8784 - Val Haversine Loss (m): 4288.7049 (Best Val: 3393.0751)
🔥 New best model saved to data/lstm_best_with_5min.pth! Val Haversine Loss (m) = 2111.6975
Epoch 3/10 - Train Haversine Loss (m): 3928.2044 - Val Haversine Loss (m): 2111.6975 (Best Val: 2111.6975)
Epoch 4/10 - Train Haversine Loss (m): 3775.1920 - Val Haversine Loss (m): 3001.3424 (Best Val: 2111.6975)
Epoch 5/10 - Train Haversine Loss (m): 3501.9388 - Val Haversine Loss (m): 2244.0617 (Best Val: 2111.6975)
Epoch 6/10 - Train Haversine Loss (m): 3257.3103 - Val Haversine Loss (m): 2872.4044 (Best Val: 2111.6975)
Epoch 7/10 - Train Haversine Loss (m): 3306.2424 - Val Haversine Loss (m): 3227.5457 (Best Val: 2111.6975)
🔥 New best model saved to data/lstm_best_with_5min.pth! Val Haversin

In [11]:
# Load best model from file
checkpoint = torch.load(
    "data/lstm_best_with_5min.pth",
    map_location=device,
    weights_only=False,  # <-- add this
)

loaded_model = LSTMEncoderDecoder(
    input_dim=INPUT_DIM,
    output_dim=OUTPUT_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
).to(device)

loaded_model.load_state_dict(checkpoint["model_state_dict"])
loaded_model.eval()

loaded_scaler = checkpoint["scaler"]
loaded_input_len  = checkpoint["input_len"]
loaded_output_len = checkpoint["output_len"]

print("Model loaded successfully!")


Model loaded successfully!


In [12]:
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# --------- EVALUATE LOADED MODEL ---------
loaded_model.eval()

# Retrieve scaling constants
lat_mean = loaded_scaler["mean"]["Latitude"]
lat_std  = loaded_scaler["std"]["Latitude"]
lon_mean = loaded_scaler["mean"]["Longtitude"]
lon_std  = loaded_scaler["std"]["Longtitude"]

all_distances = []
all_loss_value = 0.0 # Renamed from all_mse
n_samples = 0

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)

        preds = loaded_model(X_batch, Y_batch, teacher_forcing_ratio=0.0)

        # The loss is now the mean Haversine distance in meters
        loss = criterion(preds, Y_batch)
        all_loss_value += loss.item() * X_batch.size(0)
        n_samples += X_batch.size(0)

        preds_np = preds.cpu().numpy()
        true_np  = Y_batch.cpu().numpy()

        # Denormalize (Still needed for the per-step distance breakdown)
        true_lat = true_np[..., 0] * lat_std + lat_mean
        true_lon = true_np[..., 1] * lon_std + lon_mean
        pred_lat = preds_np[..., 0] * lat_std + lat_mean
        pred_lon = preds_np[..., 1] * lon_std + lon_mean

        # Compute Haversine distance
        d = haversine_m(true_lat, true_lon, pred_lat, pred_lon)
        all_distances.append(d)

all_distances = np.concatenate(all_distances, axis=0)
mean_per_step = all_distances.mean(axis=0)
overall_mean_haversine = all_loss_value / n_samples # This is the same as all_distances.mean()

print(f"Test Haversine Loss (Overall Mean): {overall_mean_haversine:.4f} meters")
print("Mean Haversine distance per prediction step (meters):", mean_per_step)
print(f"Overall mean Haversine distance (meters): {overall_mean_haversine:.4f}")

Test Haversine Loss (Overall Mean): 2226.3507 meters
Mean Haversine distance per prediction step (meters): [1672.5104 1690.351  1869.1471 2200.091  2664.0513 3261.961 ]
Overall mean Haversine distance (meters): 2226.3507


In [13]:
import numpy as np

FEATURES = [
    "Latitude", 
    "Longtitude", 
    "SOG", 
    "speed_mps_track", 
    "COG_sin",     # <--- Replaces COG
    "COG_cos"      # <--- New cyclical component
]

def inverse_transform_array(X, scaler):
    X = np.asarray(X)
    mean = np.array([scaler["mean"][f] for f in FEATURES], dtype=float)
    std  = np.array([scaler["std"][f]  for f in FEATURES], dtype=float)
    return X * std + mean


In [14]:
import folium
import itertools
import torch
import numpy as np 

# NOTE: FEATURES list must be correct (6 features)
FEATURES = ["Latitude", "Longtitude", "SOG", "speed_mps_track", "COG_sin", "COG_cos"]
LAT_IDX = FEATURES.index("Latitude")
LON_IDX = FEATURES.index("Longtitude")


def inverse_transform_array(X, scaler):
    X = np.asarray(X)
    mean = np.array([scaler["mean"][f] for f in FEATURES], dtype=float)
    std  = np.array([scaler["std"][f]  for f in FEATURES], dtype=float)
    return X * std + mean


def plot_predicted_vs_true_folium(
    model,
    test_loader,
    device,
    scaler,
    batch_idx=0,
    sample_idx=0,
    save_path=None,
):
    """
    PLOTS on Folium:
      - Predicted path (History + Prediction) in RED.
      - True path (History + True Future) in GREEN (dashed).
    """

    model.eval()

    # -------- 1. Get one batch from test_loader and Denormalize --------
    batch = next(itertools.islice(test_loader, batch_idx, batch_idx + 1))
    X_batch, Y_batch = batch

    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)

    with torch.no_grad():
        preds = model(X_batch, Y_batch, teacher_forcing_ratio=0.0)

    X_sample = X_batch[sample_idx].cpu().numpy() # (L, 6)
    Y_true_sample = Y_batch[sample_idx].cpu().numpy() # (H, 5)
    Y_pred_sample = preds[sample_idx].cpu().numpy() # (H, 5)


    # -------- 2. DENORMALIZATION LOGIC (FIXED) --------
    
    # Denormalize history (Always size 6)
    hist_denorm = inverse_transform_array(X_sample, scaler)
    hist_lat = hist_denorm[:, LAT_IDX]
    hist_lon = hist_denorm[:, LON_IDX]
    
    # -----------------------------------------------------------------
    # Case 1: Output is 5 features (Multi-Task Learning)
    # -----------------------------------------------------------------
    if Y_true_sample.shape[1] == 5: 
        
        # Denormalize future ground truth (We only need Lat/Lon, indices 0, 1)
        # Create a temporary 6-column array (Lat, Lon, SOG, 0, sin, cos) for denorm lookup
        true_stack = np.zeros((Y_true_sample.shape[0], len(FEATURES)))
        # Map Y_true features (size 5) into the 6-column stack:
        true_stack[:, [0, 1, 2, 4, 5]] = Y_true_sample[:, [0, 1, 2, 3, 4]]
        true_denorm = inverse_transform_array(true_stack, scaler)
        true_lat = true_denorm[:, LAT_IDX]
        true_lon = true_denorm[:, LON_IDX]

        # Denormalize future predictions
        pred_stack = np.zeros((Y_pred_sample.shape[0], len(FEATURES)))
        # Map prediction features (size 5) into the 6-column stack
        pred_stack[:, [0, 1, 2, 4, 5]] = Y_pred_sample[:, [0, 1, 2, 3, 4]]
        pred_denorm = inverse_transform_array(pred_stack, scaler)
        pred_lat = pred_denorm[:, LAT_IDX]
        pred_lon = pred_denorm[:, LON_IDX]

    # -----------------------------------------------------------------
    # Case 2: Output is 2 features (Position-Only Learning) - Fallback from original code
    # -----------------------------------------------------------------
    elif Y_true_sample.shape[1] == 2:
        
        # Denormalize future ground truth
        true_stack = np.zeros((Y_true_sample.shape[0], len(FEATURES)))
        true_stack[:, :2] = Y_true_sample  
        true_denorm = inverse_transform_array(true_stack, scaler)
        true_lat = true_denorm[:, LAT_IDX]
        true_lon = true_denorm[:, LON_IDX]

        # Denormalize future predictions
        pred_stack = np.zeros((Y_pred_sample.shape[0], len(FEATURES)))
        pred_stack[:, :2] = Y_pred_sample
        pred_denorm = inverse_transform_array(pred_stack, scaler)
        pred_lat = pred_denorm[:, LAT_IDX]
        pred_lon = pred_denorm[:, LON_IDX]
        
    else:
        raise ValueError(f"Unexpected Y shape {Y_true_sample.shape}; Cannot denormalize.")


    # -------- 3. Center of map --------
    all_lats = np.concatenate([hist_lat, true_lat, pred_lat])
    all_lons = np.concatenate([hist_lon, true_lon, pred_lon])

    center_lat = float(all_lats.mean())
    center_lon = float(all_lons.mean())

    m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

    # ------------------------------------------------------------------
    # *** FULL CONTINUOUS TRAJECTORY PLOTTING ***
    # ------------------------------------------------------------------
    
    # Combine History (Input Sequence) and Predicted Future (Output Sequence)
    full_predicted_lat = np.concatenate([hist_lat, pred_lat])
    full_predicted_lon = np.concatenate([hist_lon, pred_lon])

    # Combine History (Input Sequence) and True Future (Ground Truth)
    full_true_lat = np.concatenate([hist_lat, true_lat])
    full_true_lon = np.concatenate([hist_lon, true_lon])
    
    
    # -------- 5. Plot True Full Trajectory (GREEN, dashed) --------
    folium.PolyLine(
        list(zip(full_true_lat, full_true_lon)),
        popup="True Full Trajectory",
        tooltip="True Full Trajectory (History + True Future)",
        color='green', 
        weight=4,
        dash_array='5, 5' 
    ).add_to(m)
    # Mark the final true point
    folium.CircleMarker(
        location=[true_lat[-1], true_lon[-1]],
        radius=5,
        color='green',
        fill=True,
        fill_color='green',
        tooltip='True Final Position'
    ).add_to(m)


    # -------- 6. Plot Predicted Full Trajectory (RED, solid) --------
    folium.PolyLine(
        list(zip(full_predicted_lat, full_predicted_lon)),
        popup="Predicted Full Trajectory",
        tooltip="Predicted Full Trajectory (History + Prediction)",
        color='red', 
        weight=4,
    ).add_to(m)
    # Mark the final predicted point
    folium.CircleMarker(
        location=[pred_lat[-1], pred_lon[-1]],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        tooltip='Predicted Final Position'
    ).add_to(m)
    
    # Mark the divergence point (where History ends / Prediction begins)
    folium.CircleMarker(
        location=[hist_lat[-1], hist_lon[-1]],
        radius=5,
        color='black',
        fill=True,
        fill_color='yellow',
        tooltip='Prediction Divergence Point'
    ).add_to(m)


    if save_path is not None:
        m.save(save_path)
        print(f"Map saved to {save_path}")

    return m

In [15]:
m = plot_predicted_vs_true_folium(
    model=loaded_model,
    test_loader=test_loader,
    device=device,
    scaler=loaded_scaler,
    batch_idx=0,
    sample_idx=0,
    save_path="pred_vs_true_with_5min.html",
)


Map saved to pred_vs_true_with_5min.html


In [16]:
import numpy as np

# These variables (all_distances, true_np, preds_np, loaded_scaler, overall_mean_haversine)
# must be available from the previous evaluation cell (Cell 71).

# --- 1. Retrieve necessary scaling constants ---
lat_mean = loaded_scaler["mean"]["Latitude"]
lat_std = loaded_scaler["std"]["Latitude"]
lon_mean = loaded_scaler["mean"]["Longtitude"]
lon_std = loaded_scaler["std"]["Longtitude"]
sog_mean = loaded_scaler["mean"]["SOG"]
sog_std = loaded_scaler["std"]["SOG"]

# --- 2. Denormalize SOG and Lat/Lon ---
# Denormalize Lat/Lon (indices 0 and 1)
true_lat = true_np[..., 0] * lat_std + lat_mean
pred_lat = preds_np[..., 0] * lat_std + lat_mean
true_lon = true_np[..., 1] * lon_std + lon_mean
pred_lon = preds_np[..., 1] * lon_std + lon_mean

# Denormalize SOG (index 2)
true_sog = true_np[..., 2] * sog_std + sog_mean
pred_sog = preds_np[..., 2] * sog_std + sog_mean


# --- 3. CALCULATE METRICS ---

# A. Haversine RMSE (Root Mean Square Error)
haversine_rmse = np.sqrt(np.mean(all_distances**2))

# B. MAE Latitude / MAE Longitude (in degrees)
mae_lat = np.mean(np.abs(true_lat - pred_lat))
mae_lon = np.mean(np.abs(true_lon - pred_lon))

# C. Speed Error (Mean Absolute Error on SOG, in physical units)
mae_sog = np.mean(np.abs(true_sog - pred_sog))

# D. Directional Vector Error (MSE on COG_sin/cos vector)
# This serves as the proxy for Turn-Rate Error
# COG_sin/cos are at indices 3 and 4 in the 5-feature output array
true_dir = true_np[..., 3:5]
pred_dir = preds_np[..., 3:5]
dir_error_mse = np.mean((true_dir - pred_dir)**2)


# --- 4. PRINT RESULTS ---
print("=" * 40)
print("COMPREHENSIVE TEST METRICS")
print("=" * 40)
print(f"1. Mean Haversine Distance: {overall_mean_haversine:.4f} meters")
print(f"2. Haversine RMSE: {haversine_rmse:.4f} meters")
print(f"3. MAE Latitude: {mae_lat:.6f} degrees")
print(f"4. MAE Longitude: {mae_lon:.6f} degrees")
print(f"5. MAE SOG (Speed Error): {mae_sog:.4f} units")
print(f"6. Directional Vector MSE (Turn-Rate Proxy): {dir_error_mse:.6f}")

COMPREHENSIVE TEST METRICS
1. Mean Haversine Distance: 2226.3507 meters
2. Haversine RMSE: 2770.6143 meters
3. MAE Latitude: 0.016704 degrees
4. MAE Longitude: 0.013238 degrees
5. MAE SOG (Speed Error): 1.6683 units
6. Directional Vector MSE (Turn-Rate Proxy): 1.555829
